In [ ]:
ls

# inception v2 분류 모델

환경 세팅

In [ ]:
pip install setuptools==44.0.0 tensorflow==1.14.0 Cython contextlib2 matplotlib pillow lxml gast==0.2.2

# 모델 라이브러리와 데이터 다운로드

딥러닝에 필요한 models 폴더를 git 툴을 이용하여 다운로드 받습니다.

In [ ]:
!git clone https://github.com/tensorflow/models/

In [ ]:
cd models

In [ ]:
!git checkout faf5ad73be910e0010ec57b9fdd122976e8a9195

In [ ]:
cd ./research/slim

models 에서 제공하는 python 스크립트를 이용하여 분류 테스트를 위한 flower데이터를 다운로드 받습니다. 

In [ ]:
!python download_and_convert_data.py \
    --dataset_name=flowers \
    --dataset_dir=../../../flowers

다운로드 받은 데이터는 tfrecord 라는 tensorflow 에서 사용하는 데이터 형태로 지정된 폴더에 저장되어 있는 것을 확인할 수 있습니다.

In [ ]:
ls ../../../flowers

# 네트워크 훈련

훈련을 위해 잘 훈련된 모델을 다운로드

In [ ]:
cd /content/

In [ ]:
!mkdir ckp

In [ ]:
ls

In [ ]:
cd ckp

In [ ]:
!wget http://download.tensorflow.org/models/inception_v1_2016_08_28.tar.gz

In [ ]:
ls

In [ ]:
!tar xzf inception_v1_2016_08_28.tar.gz

In [ ]:
ls

훈련을 위한 준비

In [ ]:
cd /content/

In [ ]:
mkdir traindata0

In [ ]:
cd /content/models/research/slim/

훈련을 시작하기 전에 좌측 폴더 아이콘을 클릭하여 폴더 트리에서 /content/models/research/slim 폴더를  찾은 후 train_image_classifier.py 를 찾아서 더블클릭으로 열어준 후 다음과 같이 코드를 수정해준다.(책 내용 참조)

* train_image_classifier.py 592 line 에 다음과 같이 수정

-------------------------

vim train_image_classifier.py

    session_config = tf.ConfigProto(allow_soft_placement=True)

    slim.learning.train(
        train_tensor,
        logdir=FLAGS.train_dir,
        master=FLAGS.master,
        is_chief=(FLAGS.task == 0),
        init_fn=_get_init_fn(),
        summary_op=summary_op,
        number_of_steps=FLAGS.max_number_of_steps,
        log_every_n_steps=FLAGS.log_every_n_steps,
        save_summaries_secs=FLAGS.save_summaries_secs,
        save_interval_secs=FLAGS.save_interval_secs,
        sync_optimizer=optimizer if FLAGS.sync_replicas else None,
        session_config = session_config,
        )

-----------------------------------


# 훈련 시작

In [ ]:
!python train_image_classifier.py \
    --train_dir=/content/traindata0 \
    --dataset_name=flowers \
    --dataset_split_name=train \
    --dataset_dir=/content/flowers \
    --model_name=inception_v1 \
    --max_number_of_steps=1000 \
    --checkpoint_path=/content/ckp/inception_v1.ckpt \
    --checkpoint_exclude_scopes=InceptionV1/Logits \
    --trainable_scopes=InceptionV1/Logits \
    --batch_size=32 \
    --learning_rate=0.01 \
    --learning_rate_decay_type=fixed \
    --save_interval_secs=100 \
    --log_every_n_steps_secs=100 \
    --optimizer=rmsprop \
    --weight_decay=0.00004


In [ ]:
ls /content/

생성된 체크포인트 확인

In [ ]:
ls /content/traindata0/

#분류 모델 테스트

모델 평가

In [ ]:
!python eval_image_classifier.py \
    -–alsologtostderr \
    --checkpoint_path=/content/traindata1\
    --dataset_dir=/content/flowers\
    --dataset_name=flowers \
    --dataset_split_name=validation \
    --model_name=inception_v1
